# Notebook 5: Visualizing DNA replication profiles

### by Justin B. Kinney

In this exercise we will use Pandas to analyze the DNA replication origin data that we mapped earlier today.

In [ ]:
# Always put this first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# We will later need this
import os.path 

# Set seaborn style
sns.set_style('white')

The files we want are the X1.pileup.bed files. Let's use 'cat' to see what these files look like

In [ ]:
# Warning: won't work on Windows machines
!head A1.pileup.bed

This is a tab-delimited text file. This is exactly the kind of file that Pandas is designed to work with. Let's open the file up using pandas

In [ ]:
# Read file into a pandas "data frame"
df = pd.read_csv('A1.pileup.bed',  
                 names=['chromosome','start','stop','reads'], 
                 delim_whitespace=True, 
                 skiprows=2)

# Show dataframe shape
print(df.shape)

In [ ]:
# Show first 5 rows
df.head()

In [ ]:
# Check the last 5 rows of the dataframe
df.tail()

In [ ]:
# Extract values for chromosome IV only. 
chrom_we_want = 'chrIV'
flags = (df['chromosome']==chrom_we_want)
chr4_df = df[flags]
print('chr4_df has %d rows'%len(chr4_df))
chr4_df.head()

In [ ]:
# Compute read length
read_length = df.loc[0,'stop'] - df.loc[0,'start'] + 1
print(read_length)

Now let's plot these number of reads across the chromosome.

In [ ]:
# Extract vectors of starts, stops, and number of reads
starts = chr4_df['start']
stops = chr4_df['stop']
num_reads = chr4_df['reads']

# Plot the starts
plt.plot(starts, num_reads)

Well, this kinda looks right. But we're too far away to see what's going on well. Let's zoom in

In [ ]:
# Zoom in to region between 400K and 500k
zoom_indices = (starts > 4E5) & (starts < 5E5)
plt.plot(starts[zoom_indices], num_reads[zoom_indices])

It's becomming clear that, in order to make sense of this data, we're going to need to smooth it. How do we do that? Fortunately, SciPy contains tons of routines for signal processing like this. 

In [ ]:
# Create a "kernel" with which to smooth the data
# from experience, 6 kb is a nice window size
window_bp = 6000
read_length = 31
window_size = window_bp//read_length
print(window_size)

In [ ]:
# Create a window kernel so that all elements sum to 1
window = np.ones(window_size)/window_size

# Make sure that the window is properly normalized
sum(window)

In [ ]:
from scipy.signal import convolve

# Smooth the data by convolving the window with the read counts
smooth_reads = convolve(num_reads,window,'same')

In [ ]:
plt.plot(starts[zoom_indices], num_reads[zoom_indices])
plt.plot(starts[zoom_indices], smooth_reads[zoom_indices])

Hm, this is a bit hard to see. Let's clairfy the plot.

In [ ]:
plt.plot(starts[zoom_indices], num_reads[zoom_indices], color='orange')
plt.plot(starts[zoom_indices], smooth_reads[zoom_indices], color='lightblue', linewidth=3)

Great. Now let's make a chromosome-wide replication profile. Let's also put a little effort into making it look pretty. 

In [ ]:
# Easier to discuss genomic positions in kb
kb = 1E3
x = 0.5*(starts+stops)/kb
L = max(x)

figure_size = [20,2.5]
label_size = 16
title_size = 20
plt.figure(figsize=figure_size)

# Plot
plt.fill_between(x, smooth_reads, color='black')

# Place tick marks on x axis ever 200 kb
plt.xticks(np.arange(0, L, 200), fontsize=label_size)

# Fit plot to precisely the width of the chromosome
plt.xlim([min(x), max(x)])

# Choose ylims too
plt.ylim(0,max(smooth_reads*1.2))

# Turn off box
plt.box(False)

# No need to show ticks on the y axes
plt.yticks([])

# Add some text annotation, telling the user
plt.title('DNA replication profile', fontsize=title_size)
plt.xlabel(chrom_we_want + ' position (kb)', fontsize=label_size)

# Fix spacing
plt.tight_layout()

file_name = '5_dnareplication_1.png'
plt.savefig(file_name)

plt.show()

Great! With this smoothed data we can go ahead and identify peaks in the replicaiton profile, measure their width, height, etc. 

In [ ]:
# WARNING: Won't work on windows machines
!ls -lah

In [ ]:
!open $file_name

Now let's write a function that will load and smooth the profile from any chromosome we want

In [ ]:
def load_and_smooth_profile(file_name, chrom_we_want, window_bp=6000):
    """
    Load a smooth a replication profile.
    
    Arguments:
        file_name -- name of a bed file containing the data
        chrom_we_want -- chromosome name in Roman numerals: chrI, ..., chrXVI
        window_bp -- length of smoothing window in bp
    
    Returns:
        centers -- location of bin centers along the chromosome
        smooth_reads -- smoothed number of reads in each bin
    """
   
    # Check validity of file
    assert isinstance(file_name,str), 'file_name is not a string.'
    assert os.path.isfile(file_name), 'file %s does not exist'%file_name
    
    # Read file into dataframe 
    df = pd.read_csv(file_name,  
                 names=['chromosome','start','stop','reads'], 
                 delim_whitespace=True, 
                 skiprows=2)
    
    # Check validity of chromosome
    assert isinstance(chrom_we_want, str), 'chrom_we_want is not a string'
    assert chrom_we_want in set(df['chromosome']), \
        'chrom_we_want value %s not found in data frame.'%chrom_we_want
        
    # Check validity of window_bp
    assert isinstance(window_bp, int), "window_bp is not an integer."
    assert window_bp > 0, "nonpositive window_bp value of %d"%window_bp
    
    # Choose rows to look at
    indices = (df['chromosome']==chrom_we_want)
    
    # Extract read_lenght, num_reads and centers
    chr_df = df[indices]
    num_reads = chr_df['reads']
    starts = chr_df['start']
    stops = chr_df['stop']
    centers = 0.5*(starts + stops)
    
    # Create convolution window
    read_length = df.loc[0,'stop'] - df.loc[0,'start'] + 1
    window_size = window_bp//read_length
    window = np.ones(window_size)/window_size
    
    # Smooth read counts
    smooth_reads = convolve(num_reads,window,'same')
    
    # Now return results
    return centers, smooth_reads

In [ ]:
def plot_profile(centers, 
                 smooth_reads,  
                 title='', 
                 kbspacing=200, 
                 color=[0, .5, 1]):
    """
    Plot a replication profile
    
    Arguments:
        centers -- numpy array of bin locations
        smooth_reads -- numpy array of smoothed read counts; same length as centers
        title -- title for plot
        kbspacing -- tick mark spaking in kilobases
        color -- color of plot
    """
    
    assert len(centers)==len(smooth_reads), "centers and smooth_reads are not the same length."
    assert isinstance(title,str), "title is not a string"
    assert isinstance(kbspacing,int), "kbspacing is not an int"
    assert kbspacing > 0, "nonpositive kbspacing value of %d"%kbspacing
    
    # Easier to discuss genomic positions in kb
    kb = 1E3
    x = centers/kb
    L = max(x)
    
    label_size = 16
    
    # Plot
    plt.fill_between(x, smooth_reads, color=color)

    # Place tick marks on x axis ever 200 kb
    plt.xticks(np.arange(0,L,kbspacing), fontsize=label_size)
    
    # Fit plot to precisely the width of the chromosome
    plt.xlim([min(x), max(x)])
    
    # Turn off box
    plt.box(False)
    
    # No need to show ticks on the y axes
    plt.yticks([])
    
    # Add some text annotation, telling the user
    plt.title(title, fontsize=label_size)
    plt.xlabel('position (in kb)', fontsize=label_size)
    

In [ ]:
# Test these functions
plt.figure(figsize=[10,2])
ctrs, rds = load_and_smooth_profile('A1.pileup.bed', chrom_we_want)
plot_profile(ctrs, rds, title='sample A', color='green')
plt.tight_layout()

## Exercise

**E5.1.** Using `load_and_smooth_profile()` and `plot_profile()`, fill in code in the subsequent cell to plot four replication profiles in a single figure, using the specified colors.

In [ ]:
# Plot 4 replication profiles
plt.figure(figsize=[20,5])

# Chromosome to focus on
chrom_we_want = 'chrII'

# Names of samples
samples = ['A','B','C','D']

# Define colors
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']

# Look over each sample
for i, (sample, color) in enumerate(zip(samples, colors)):

    ### Answer here -- Specify file name, then load centers and smoothed counts
    
    # Create axes on which to plot 
    plt.subplot(2,2,i+1)
    
    ### Answer here -- Plot profile here

# Fix layout
plt.tight_layout(w_pad=5, h_pad=2)

# Save 
file_name = '5_dnareplication_2.png'
plt.savefig(file_name)

# Show plot
plt.show()

# Open figure in another applicaiton
!open $file_name

Done.